In [1]:
import sys
import glob
import os
import pandas as pd
import ee
import numpy as np
import geopandas as gp

import matplotlib.pyplot as plt
import rsfuncs as rs

from scipy import stats
from datetime import datetime, timedelta
from datetime import datetime as dt

%load_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = [18,16]
ee.Initialize()

In [2]:
# EE functions are in the rsfuncs module. Loacal functions are here: 

In [85]:
def read_sm_file(filename):
    with open(filename) as f:
        contents = f.readlines()

    data = []
    for line in contents:
        if line[0:1] == "#":
            continue
        else:
            data.append(line)

    headers = [x.replace("Soil Moisture Percent","smp").replace(" ","_") for x in data[0].split(",")]
    cols = [x.strip("\n").split(",") for x in data[1:]]

    df = pd.DataFrame(cols, columns = headers)
    
    return df

def get_ims_by_date(ims_list, var, res=10):
    imlist = []
    imdates = []
    num_images = len(ims_list.getInfo())

    for i in range (0, num_images):
        
        if i % 5 == 0:
            print(str((i / num_images)*100)[:5] + " % ")

        latlon = ee.Image.pixelLonLat().addBands(ims_list.get(i))
        imlist.append(array_from_latlon(latlon, var, res))
        date =  latlon.get('system:time_start')
        info_dict = ims.get(i).getInfo()
        date = info_dict['id']
        imdates.append(date)

    return imlist, imdates

def array_from_latlon(latlon_obj, var, res ):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.toList(),geometry=area,maxPixels=1e8,scale=res)
    try:
        lats = np.array((ee.Array(res.get("latitude")).getInfo()))
        lons = np.array((ee.Array(res.get("longitude")).getInfo()))
        data = np.array((ee.Array(res.get(var)).getInfo()))
    except:
        data = np.full_like(lats, np.nan,dtype=np.float64)
    
    out = make_np_array(data, lats, lons)
    return out   

def make_np_array(data, lats, lons):
    # get data from df as arrays
    lons = np.array(lons)
    lats = np.array(lats)
    data = np.array(data) # Set var here 
                                              
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)

    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)

    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]

    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32)

    # fill the array with values
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner
    
    return arr

def filter_date(product,y,m,d):
    start = ee.Date.fromYMD(y,m,d).advance(-1, "day")
    end = ee.Date.fromYMD(y,m,d)
    prod = product.filterDate(start, end).sort('system:time_start', False).select("ppt")
    return prod

def get_2day_precip(latlon_obj, area):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.sum(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("ppt")).getInfo()))
    out = np.array(data)
    return out 

def get_ndvi(latlon_obj, area):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.mean(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("NDVI")).getInfo()))
    out = np.array(data)
    return out 

In [86]:
site_file = gp.read_file("../shape/scan_sites.shp")
sites = site_file[~site_file['state'].isin(["AK", "HI", "PR"])]

In [87]:
data = rs.load_data()

In [88]:
data_dir = "../data"

In [141]:
dfs = []
lctypes = []

In [ ]:
# For each site id, find the sm file with the data (in data dir), 
# calculate psi as psi = A * SM ^b (krishna's paper )
# query the (1) landcover, (2) Sentinel backscatter (Prism P), MODIS / Landsat LAI for the whole timeseries 
   
for idx, row in sites[24:].iterrows():
    
    print("Processings site no {}".format(row.id))
    
    # Make geom to submit to EE 
    x,y = row.geometry.buffer(0.0001).envelope.exterior.coords.xy
    coords = [list(zip(x,y))]
    area = ee.Geometry.Polygon(coords)
    
    # Get the corresponding SCAN data file from data folder
    site_id = row.id
    sm_file = [os.path.join(data_dir,x) for x in os.listdir(data_dir) if site_id in x][0]
    sm_dat = read_sm_file(sm_file)
    sm_dat['Date'] =  pd.to_datetime(sm_dat['Date'], format='%Y%m%d %H:%M')
    sm_dat.set_index('Date', inplace=True)
        
    # start and end date
    startdate = sm_dat.index[0]
    enddate = sm_dat.index[-1]
    
    date = startdate.strftime("%Y-%m-%d")
    
    # Select the nlcd dataset
    dataset =  rs.load_data()['nlcd']
    ic = dataset[0]
    var = dataset[1]
    res = dataset[3]
    
    # find the nearest nlcd dataset 
    col = ic.filterDate(ee.Date(date).advance(-1, 'years'), ee.Date(date).advance(2, 'years')).first()
    t = col.reduceRegion(ee.Reducer.frequencyHistogram(), area, res).get(var)
    meta = col.getInfo()
    lc_class = int(list(t.getInfo().keys())[0])
    
    # Get the landcover type
    lcidx = meta['properties']['landcover_class_values'].index(lc_class)
    lctype = meta['properties']['landcover_class_names'][lcidx]
    print(lctype)
    lctypes.append(lctype)
    
    # Get Sentinel images and dates (ascending orbits only, VV polarization)
    s1 = rs.load_data()['s1']
    s1ic, s1var, s1res = s1[0], s1[1], s1[3]
    
    s1_col = s1ic.filterBounds(area).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    s1_col = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).select(s1var)
    
    # Krishna used ascending pass... 
    
    s1_col = s1_col.filterDate(startdate,enddate)
    t = s1_col.sort('system:time_start')
    ims = t.toList(t.size())
    
    print("Processing {} sentinel overapasses".format(len(ims.getInfo())))

    s1dat, dates = get_ims_by_date(ims,s1var)
    
    # Calc the S1 mean backscatter in each image
    mean_sigmas = []
    std_sigmas = []

    for i in s1dat:
        mean_sigmas.append(np.mean(i))
        std_sigmas.append(np.std(i))
        
    # Convert the datestrings from S1 to pandas datetimes 
    for idx, x in enumerate(dates):
        timestamp = x.find("V_")+2
        timestr = x[timestamp:timestamp+13]
        dates[idx] = pd.to_datetime(timestr, format='%Y%m%d %H:%M')
        
    # Get PRISM data for all the S1 overpass dates to filter the rainy days
    print("processing PRISM")
    rainfall = []

    for i,x in enumerate(dates):
        if i % 5 == 0:
            print(str((i / len(dates))*100)[:5] + " % ")
        
        y,m,d = dates[i].year, dates[i].month, dates[i].day
        t = filter_date(rs.load_data()['prism_daily'][0], y, m, d).sum()
        precip_total = get_2day_precip(t, area)
        rainfall.append(precip_total)

    
    # MODIS LAI 
    print("processing MODIS")
    modis = rs.load_data()['modis_lai']
    mic, mvar, mres = modis[0],modis[1], modis[2]

    modis_lai = []
    for i,x in enumerate(dates):
        if i % 5 == 0:
            print(str((i / len(dates))*100)[:5] + " % ")
            
        y,m,d = dates[i].year, dates[i].month, dates[i].day

        start = ee.Date.fromYMD(y,m,d).advance(-2,"day")
        end = ee.Date.fromYMD(y,m,d).advance(2, "day")

        prod = mic.filterDate(start, end).sort('system:time_start', False).select(mvar).first()

        try:
            res = prod.reduceRegion(reducer=ee.Reducer.first(),geometry=area,scale=5) # Need to have scale smaller than geom
            data = np.array((ee.Array(res.get(mvar)).getInfo()))
            out = np.array(data)* 0.1
            modis_lai.append(out)
        except:
            out = np.nan
            modis_lai.append(np.nan)
                        
    # Filter the SCAN data for the S1 dates, 3 am - 7 am 
    rzsm = []
    ssm = []
    
    for i in dates:
        starttime = i.replace(second=0, microsecond=0, minute=0, hour=3)
        endtime = starttime+timedelta(hours= 4)
        df = pd.DataFrame(sm_dat[starttime:endtime])
        
        rzsm.append(df[df.columns[-1]].values)
        ssm.append(df[df.columns[-3]].values)
    
    # In case there are nans or data gaps in the sm data
    rzsm = [list(filter(None, x)) for x in rzsm]
    ssm = [list(filter(None, x)) for x in ssm]
    
    # Calc the 5 hour mean for each sensor reading 
    for idx,x in enumerate(rzsm):
        rzsm[idx] = np.nanmean([np.float(i) for i in x])
    
    for idx,x in enumerate(ssm):
        ssm[idx] = np.nanmean([np.float(i) for i in x])

    # params to calculate psi 
    a = row.a
    b = row.b
    
    # RZ and Surface LWP = A * sm ^b
    psi1 = [a*(x/100)**b for x in rzsm]
    psi2 = [a*(x/100)**b for x in ssm]
    
    df = pd.DataFrame([dates, mean_sigmas, std_sigmas, rainfall, modis_lai, rzsm,psi1,ssm,psi2])
    df = df.T
    df.columns = (['date', "sigma", "std_sigma", "precip","LAI", "rzsm","psi_rz", "ssm", "psi_s"])
    mask=(df['precip'] < 0.1)
    dfout = df[mask]
    
    dfs.append(dfout)
    
    print(dfout.head())
    

Processings site no 2026
Developed, Open Space - Includes areas with a mixture of some constructed materials, but mostly vegetation in the form of lawn grasses.  Impervious surfaces account for less than 20 percent of total cover.  These areas most commonly include large-lot single-family housing units, parks, golf courses, and vegetation planted in developed settings for recreation, erosion control, or aesthetic purposes.
Processing 82 sentinel overapasses
0.0 % 
6.097 % 
12.19 % 
18.29 % 
24.39 % 
30.48 % 
36.58 % 
42.68 % 
48.78 % 
54.87 % 
60.97 % 
67.07 % 
73.17 % 
79.26 % 
85.36 % 


In [161]:
    for idx, x in enumerate(dates):
        print(x)
        timestamp = x.find("V_")+2
        timestr = x[timestamp:timestamp+13]
        print(timestr)

COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20150729T131943_20150729T132010_007026_0098BC_7288
20150729T1319
COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20150822T131944_20150822T132011_007376_00A23E_511C
20150822T1319
COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20151009T131945_20151009T132012_008076_00B516_C9B6
20151009T1319
COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20151102T131945_20151102T132012_008426_00BE82_E35C
20151102T1319
COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20151126T131939_20151126T132006_008776_00C82D_D34C
20151126T1319
COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20151220T131938_20151220T132005_009126_00D1F8_FC5B
20151220T1319
COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20160113T131938_20160113T132004_009476_00DBF9_CA69
20160113T1319
COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20160206T131937_20160206T132004_009826_00E623_EA16
20160206T1319
COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20160301T131937_20160301T132004_010176_00F040_5361
20160301T1319
COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20160418T131938_20160418T132005_010876_01049B_6831
20160418T1319


In [155]:
dates

['COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20150729T131943_20150729T132010_007026_0098BC_7288',
 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20150822T131944_20150822T132011_007376_00A23E_511C',
 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20151009T131945_20151009T132012_008076_00B516_C9B6',
 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20151102T131945_20151102T132012_008426_00BE82_E35C',
 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20151126T131939_20151126T132006_008776_00C82D_D34C',
 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20151220T131938_20151220T132005_009126_00D1F8_FC5B',
 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20160113T131938_20160113T132004_009476_00DBF9_CA69',
 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20160206T131937_20160206T132004_009826_00E623_EA16',
 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20160301T131937_20160301T132004_010176_00F040_5361',
 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20160418T131938_20160418T132005_010876_01049B_6831',
 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SSV_20160512T131942_20160512T132009_011226_010F9F_7EE5',
 'COPERNIC

In [ ]:
x = np.array(mean_sigmas)
x = x.reshape(x.shape[0])
y = np.array(rzsm)
y = y.reshape(y.shape[0])
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

In [ ]:
plt.figure(figsize=(8,6))

plt.title('''$\sigma$ vs Soil moisture 
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], "10"))

plt.xlabel("Backscatter ($\sigma$), dB")
plt.scatter(mean_sigmas, rzsm, label = 'rzsm')
plt.scatter(mean_sigmas, ssm, label = 'ssm')
plt.plot(x, intercept + slope*x, 'blue', label='rzsm y = {}x +{}, $r =$ {}'.format(round(slope,1), round(intercept,1), round(r_value,2)))
plt.xlabel("$\sigma$ (dB)")
plt.ylabel("Soil moisture (%)")
plt.legend()
plt.show()

In [ ]:
# as a timeseries

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,6))

plt.title('''$\sigma$ vs Soil moisture Time Series
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], buffer_size))

color = 'tab:red'
ax1.set_xlabel('date')
ax1.set_ylabel("$\sigma$ (dB)", color=color)
ax1.plot(dates, mean_sigmas, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('rzsm', color=color)  
ax2.plot(dates,rzsm,color = color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout() 

plt.show()

In [ ]:
# remove the days with high preceding rainfall 

In [ ]:
imageCollection = ee.ImageCollection("OREGONSTATE/PRISM/AN81d").filterBounds(area)

In [ ]:
def filter_date(product,y,m,d):
    start = ee.Date.fromYMD(y,m,d).advance(-1, "day")
    end = ee.Date.fromYMD(y,m,d)
    end = ee.Date.fromYMD(y,m,d)
    prod = product.filterDate(start, end).sort('system:time_start', False).select("ppt")
    return prod

def get_2day_precip(latlon_obj):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.sum(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("ppt")).getInfo()))
    out = np.array(data)
    return out 

In [ ]:
rainfall = []

for i,x in enumerate(dates):
    y,m,d = dates[i].year, dates[i].month, dates[i].day
    t = filter_date(imageCollection, y, m, d).sum()
    precip_total = get_2day_precip(t)
    rainfall.append(precip_total)

In [ ]:
fdf = pd.DataFrame([dates, mean_sigmas, rzsm, rainfall])

In [ ]:
fdf = fdf.T
fdf.columns = (['date', "sigma", "rzsm", "rainfall"])

In [ ]:
mask=(fdf['rainfall'] < 0.1)
asdf=fdf[mask]

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,6))

plt.title('''2 day sum Precip (mm) and Soil moisture (%) Time Series
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], buffer_size))

color = 'tab:blue'
ax1.set_xlabel('date')
ax1.set_ylabel("Precip (mm)", color=color)
ax1.plot(dates, rainfall, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('rzsm', color=color)  
ax2.plot(dates,rzsm,color = color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout() 

plt.show()

In [ ]:
# Plot linear fit of sigma vs SM
x = np.array(asdf['sigma'])
x = x.reshape(x.shape[0]).astype(np.float32)
y = np.array(asdf['rzsm'])
y = y.reshape(y.shape[0]).astype(np.float32)

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

plt.figure(figsize=(8,6))

plt.title('''$\sigma$ vs Soil moisture for overpasses without prior precipitation
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(asdf), asdf['date'].min(), asdf['date'].max(), buffer_size))

plt.xlabel("Backscatter ($\sigma$), dB")
plt.scatter(asdf['sigma'],asdf['rzsm'], label = "rzsm")
# plt.scatter(mean_sigmas, ssm, label = 'ssm')
plt.plot(x, intercept + slope*x, 'blue', label='rzsm y = {}x +{}, $r =$ {}'.format(round(slope,1), round(intercept,1), round(r_value,2)))
plt.xlabel("$\sigma$ (dB)")
plt.ylabel("Soil moisture (%)")
plt.legend()
plt.show()

In [ ]:
# Let's test with 4 day precip accumulation

imageCollection = ee.ImageCollection("OREGONSTATE/PRISM/AN81d").filterBounds(area)

def filter_date(product,y,m,d):
    start = ee.Date.fromYMD(y,m,d).advance(-4, "day")
    end = ee.Date.fromYMD(y,m,d)
    end = ee.Date.fromYMD(y,m,d)
    prod = product.filterDate(start, end).sort('system:time_start', False).select("ppt")
    return prod

def get_2day_precip(latlon_obj):
    res = latlon_obj.reduceRegion(reducer=ee.Reducer.sum(),geometry=area,scale=10)
    data = np.array((ee.Array(res.get("ppt")).getInfo()))
    out = np.array(data)
    return out 

rainfall = []

for i,x in enumerate(dates):
    y,m,d = dates[i].year, dates[i].month, dates[i].day
    t = filter_date(imageCollection, y, m, d).sum()
    precip_total = get_2day_precip(t)
    rainfall.append(precip_total)
    

In [ ]:
fdf = pd.DataFrame([dates, mean_sigmas, rzsm, rainfall])
fdf = fdf.T
fdf.columns = (['date', "sigma", "rzsm", "rainfall"])
mask=(fdf['rainfall'] < 0.1)
asdf=fdf[mask]

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,6))

plt.title('''4 day sum Precip (mm) and Soil moisture (%) Time Series
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(dates), dates[0], dates[-1], buffer_size))

color = 'tab:blue'
ax1.set_xlabel('date')
ax1.set_ylabel("3 day precipitation (mm)", color=color)
ax1.plot(dates, rainfall, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('rzsm', color=color)  
ax2.plot(dates,rzsm,color = color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout() 

plt.show()

In [ ]:
# Plot linear fit of sigma vs SM
x = np.array(asdf['sigma'])
x = x.reshape(x.shape[0]).astype(np.float32)
y = np.array(asdf['rzsm'])
y = y.reshape(y.shape[0]).astype(np.float32)

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

plt.figure(figsize=(8,6))

plt.title('''$\sigma$ vs Soil moisture for overpasses without prior precipitation (4 days)
          n_overpasses: {}  
          startdate: {} 
          enddate: {}
          buffer size: {}m'''.format(len(asdf), asdf['date'].min(), asdf['date'].max(), buffer_size))

plt.xlabel("Backscatter ($\sigma$), dB")
plt.scatter(asdf['sigma'],asdf['rzsm'], label = "rzsm")
# plt.scatter(mean_sigmas, ssm, label = 'ssm')
plt.plot(x, intercept + slope*x, 'blue', label='rzsm y = {}x +{}, $r =$ {}'.format(round(slope,1), round(intercept,1), round(r_value,2)))
plt.xlabel("$\sigma$ (dB)")
plt.ylabel("Soil moisture (%)")
plt.legend()
plt.show()